In [ ]:
!pip install selenium

In [ ]:
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys

import pandas as pd

In [ ]:
# 본인의 경로 설정해서 붙여두기
s = Service("C:/Users/USER/Documents/chromedriver/chromedriver.exe")
driver = webdriver.Chrome(service=s)
time.sleep(2)

# 윈도우 크기 조절
driver.maximize_window()

In [ ]:
# 왓챠피디아 링크 접속
driver.get('https://pedia.watcha.com/ko-KR')

## 영화 정보 크롤링

In [ ]:
movie = pd.read_excel('역대 한국 박스오피스 흥행작_장르, 키워드, 감독 포함.xlsx')
movie['개봉일'] = movie['개봉일'].map(lambda x: str(x)[:4])
movie_title = movie[['영화명','개봉일','감독']].values

In [ ]:
# 크롤링 맞춰서 제목,감독,연도 수정
movie_title[7][1] = '2014'
movie_title[27][1] = '2013'
movie_title[62][2] = '조스 웨던'
movie_title[89][2] = '애너 보든'
movie_title[103][2] = '스콧 데릭슨'
movie_title[112][2] = '토드 필립스'
movie_title[123][2] = '피트 닥터'
movie_title[139][2] = '바이론 하워드'
movie_title[141][1] = '2012'
movie_title[165][0] = '인디아나 존스: 크리스탈 해골의 왕국'
movie_title[184][2] = '루벤 플레이셔'
movie_title[187][2] = '볼프강 페터젠'
movie_title[227][0] = '다이하드4'
movie_title[206][2] = '데미언 셔젤'
movie_title[207][1] = '2015'
movie_title[212][2] = '리 언크리치'
movie_title[280][2] = '케이트 숏랜드'
movie_title[281][2] = '아니쉬 채겐티'
movie_title[303][2] = '크리스 윌리엄스'
movie_title[319][0] = '지아이조 - 전쟁의 서막'
movie_title[320][2] = '루이 르테리에'
movie_title[323][1] = '2016'
movie_title[330][2] = '크리스 샌더스'
movie_title[357][2] = '톰 맥그라스'
movie_title[369][2] = '피에르 모렐'
movie_title[408][2] = '커스틴 쉐리단'
movie_title[410][2] = '볼프강 페터젠'
movie_title[426][2] = '패티 젠킨스'
movie_title[439][2] = '데이비드 슬레이드'
movie_title[445][2] = '자움 콜렛-세라'
movie_title[458][2] = '피트 닥터'
movie_title[460][2] = '대런 애로노프스키'
movie_title[461][0] = '캐리비안의 해적 - 블랙 펄의 저주'
movie_title[490][2] = '데이비드 에이어'
movie_title[494][2] = '리처드 커티스'

In [ ]:
## 영화 url 뽑아오기

movie_url = pd.DataFrame()
titles = []
dirs = []
urls = []

for movie in tqdm(movie_title[490:]):
    if movie[0] == '스파이':
        continue
    if movie[0] == '블라인드':
        continue
    movie = list(movie)
    driver.get('https://pedia.watcha.com/ko-KR') # 왓챠피디아 링크 접속
    time.sleep(2)

    # 검색창에 입력값 보내기
    search_xpath = '/html/body/div/div/div[1]/header[1]/nav/div/div/ul/li[6]/div/div/form/label/input'
    search = driver.find_element('xpath', search_xpath)
    search.send_keys(movie[0])
    search.send_keys(Keys.ENTER)
    time.sleep(1.5)

    # 영화 더보기로 들어가기
    click_plus_xpath = '/html/body/div/div/div[1]/section/section/div[3]/div[1]/section/section[2]/div[1]/div/header/div/div/a'
    click_plus = driver.find_element('xpath', click_plus_xpath)
    click_plus.click()
    time.sleep(1.5)

    # 소스코드 BeautifulSoup에 입력하기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 감독, 연도랑 일치하는 영화
    movie_list = soup.find_all('li', {'class':'css-ix1flm'})
    if len(movie_list) == 1:
        url = movie_list[0].find('a', {'class':'css-1aaqvgs-InnerPartOfListWithImage'}).get('href')
        title1 = movie_list[0].find('div', {'class':'css-x62r3q'}).text
        dir1 = movie_list[0].find('div', {'class':'css-1b6e4cd'}).text
    else:
        director = list(map(lambda x: x.find('div', {'class':'css-1b6e4cd'}), movie_list))
        director = [x.text if x != None else 0 for x in director]
        dir_idx = [i for i,j in enumerate(director) if j == movie[2]]

        if len(dir_idx) == 1:
            dir_idx = dir_idx[0]
            url = movie_list[dir_idx].find('a', {'class':'css-1aaqvgs-InnerPartOfListWithImage'}).get('href')
            title1 = movie_list[dir_idx].find('div', {'class':'css-x62r3q'}).text
            dir1 = movie_list[dir_idx].find('div', {'class':'css-1b6e4cd'}).text
        else:
            year = list(map(lambda x: x.find('div', {'class':'css-h25two'}), movie_list))
            year = [x.text.split('・')[0].replace(' ','') if x != None else 0 for x in year]
            idx = [i for i,j in enumerate(zip(year, director)) if list(j) == movie[1:]][0]
            url = movie_list[idx].find('a', {'class':'css-1aaqvgs-InnerPartOfListWithImage'}).get('href')
            title1 = movie_list[idx].find('div', {'class':'css-x62r3q'}).text
            dir1 = movie_list[idx].find('div', {'class':'css-1b6e4cd'}).text

    urls.append(url)
    titles.append(title1)
    dirs.append(dir1)

movie_url['영화명'] = titles
movie_url['감독'] = dirs
movie_url['url'] = urls
movie_url

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:42<00:00,  6.05s/it]


,영화명,감독,url
0,수어사이드 스쿼드,데이비드 에이어,/ko-KR/contents/mWvkyor
1,헤어질 결심,박찬욱,/ko-KR/contents/m5DP0eR
2,오늘의 연애,박진표,/ko-KR/contents/mOkXVQd
3,글러브,강우석,/ko-KR/contents/mZ5e9R5
4,러브 액츄얼리,리처드 커티스,/ko-KR/contents/mBOkbkW
5,라디오 스타,이준익,/ko-KR/contents/mYdE0OX
6,퍼시잭슨과 번개 도둑,크리스 콜럼버스,/ko-KR/contents/mWJy3J5


In [ ]:
# movie_urls = pd.DataFrame()
movie_urls = pd.concat([movie_urls, movie_url])

In [ ]:
movie_urls.reset_index(drop=True, inplace=True)
movie_urls

,영화명,감독,url
0,명량,김한민,/ko-KR/contents/mWzM9D5
1,극한직업,이병헌,/ko-KR/contents/mdMBlkR
2,신과함께-죄와 벌,김용화,/ko-KR/contents/mWw8zEO
3,국제시장,윤제균,/ko-KR/contents/mdReBLO
4,어벤져스: 엔드게임,조 루소,/ko-KR/contents/mdBzNnP
...,...,...,...
489,오늘의 연애,박진표,/ko-KR/contents/mOkXVQd
490,글러브,강우석,/ko-KR/contents/mZ5e9R5
491,러브 액츄얼리,리처드 커티스,/ko-KR/contents/mBOkbkW
492,라디오 스타,이준익,/ko-KR/contents/mYdE0OX


In [ ]:
movie_urls.loc[494] = ['스파이', '폴 페이그', '/ko-KR/contents/m5rQoaV']
movie_urls.loc[495] = ['스파이', '이승준', '/ko-KR/contents/mW48noW']
movie_urls.loc[496] = ['블라인드', '안상훈', '/ko-KR/contents/m5e77oO']
movie_urls

,영화명,감독,url
0,명량,김한민,/ko-KR/contents/mWzM9D5
1,극한직업,이병헌,/ko-KR/contents/mdMBlkR
2,신과함께-죄와 벌,김용화,/ko-KR/contents/mWw8zEO
3,국제시장,윤제균,/ko-KR/contents/mdReBLO
4,어벤져스: 엔드게임,조 루소,/ko-KR/contents/mdBzNnP
...,...,...,...
492,라디오 스타,이준익,/ko-KR/contents/mYdE0OX
493,퍼시잭슨과 번개 도둑,크리스 콜럼버스,/ko-KR/contents/mWJy3J5
494,스파이,폴 페이그,/ko-KR/contents/m5rQoaV
495,스파이,이승준,/ko-KR/contents/mW48noW


In [ ]:
movie_urls.to_csv('왓챠피디아 영화 url.csv', index=False)

In [ ]:
movie_urls

,영화명,감독,url
0,명량,김한민,/ko-KR/contents/mWzM9D5
1,극한직업,이병헌,/ko-KR/contents/mdMBlkR
2,신과함께-죄와 벌,김용화,/ko-KR/contents/mWw8zEO
3,국제시장,윤제균,/ko-KR/contents/mdReBLO
4,어벤져스: 엔드게임,조 루소,/ko-KR/contents/mdBzNnP
...,...,...,...
492,라디오 스타,이준익,/ko-KR/contents/mYdE0OX
493,퍼시잭슨과 번개 도둑,크리스 콜럼버스,/ko-KR/contents/mWJy3J5
494,스파이,폴 페이그,/ko-KR/contents/m5rQoaV
495,스파이,이승준,/ko-KR/contents/mW48noW


In [ ]:
movie_name = list(movie_urls['영화명'])
movie_name

In [ ]:
movie_ID = list(movie_urls['url'].map(lambda x: 'https://pedia.watcha.com' + x + '/comments'))
movie_ID

In [ ]:
len(movie_name) == len(movie_ID)

True

In [ ]:
# 영화 평점 크롤링
movie_rating = pd.DataFrame()

for title, ID in tqdm(zip(movie_name[300:], movie_ID[300:])):
    driver.get(ID) # 영화 페이지 접속
    time.sleep(2)

    # 스크롤 다운
    body = driver.find_element('css selector', 'body')
    for rep in range(7):
        body.send_keys(Keys.END)
        time.sleep(1)

    # 소스코드 BeautifulSoup에 입력하기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 사용자ID, 평점 수집 => 평점을 안매긴 사용자는 패스
    div_list = soup.find_all('div', {'class':'css-4obf01'})
    div_rating = list(map(lambda x: x.find('div', {'class':'css-yqs4xl'}), div_list))
    del_idx = -1

    if None in div_rating:
        del_idx = [i for i,j in enumerate(div_rating) if j == None]
        for idx in del_idx[::-1]:
            del div_rating[idx]

    if del_idx != -1:
        for idx in del_idx[::-1]:
            del div_list[idx]

    rating = list(map(lambda x:x.find('span').text, div_rating))
    user_list = list(map(lambda x: x.find('a', {'class': 'css-1f9m1s4-StylelessLocalLink eovgsd01'}).get('href'), div_list))
    user_ID = list(map(lambda x: x.split('/')[-1], user_list))

    # 데이터프레임 저장
    temp = pd.DataFrame()
    temp['user'] = user_ID
    temp['movie'] = title
    temp['movie_ID'] = ID
    temp['rating'] = rating
    movie_rating = pd.concat([movie_rating, temp])
    print(len(movie_rating))
    time.sleep(1)

movie_rating.reset_index(drop=True, inplace=True)
movie_rating

0it [00:00, ?it/s]

56


1it [00:11, 11.80s/it]

110


2it [00:23, 11.75s/it]

...

10891


196it [38:12, 11.71s/it]

10947


197it [38:23, 11.69s/it]


,user,movie,movie_ID,rating
0,WwRqopwVyvlzB,해리포터와 죽음의 성물 1,https://pedia.watcha.com/ko-KR/contents/m5ZXwL...,3.5
1,87Gv7eQBjxE6o,해리포터와 죽음의 성물 1,https://pedia.watcha.com/ko-KR/contents/m5ZXwL...,4.0
2,DgwxAeQYNxrMj,해리포터와 죽음의 성물 1,https://pedia.watcha.com/ko-KR/contents/m5ZXwL...,3.0
3,nkPvrJ84Dvgar,해리포터와 죽음의 성물 1,https://pedia.watcha.com/ko-KR/contents/m5ZXwL...,5.0
4,Qgy51MPlQ5Djk,해리포터와 죽음의 성물 1,https://pedia.watcha.com/ko-KR/contents/m5ZXwL...,4.0
...,...,...,...,...
10942,JgAx89MRDqLbO,블라인드,https://pedia.watcha.com/ko-KR/contents/m5e77o...,3.0
10943,nkPvrQAPJqgar,블라인드,https://pedia.watcha.com/ko-KR/contents/m5e77o...,3.0
10944,a9L5PwKN2vwg8,블라인드,https://pedia.watcha.com/ko-KR/contents/m5e77o...,3.0
10945,24BqEMAnOqrjn,블라인드,https://pedia.watcha.com/ko-KR/contents/m5e77o...,3.0


In [ ]:
movie_rating

,user,movie,movie_ID,rating
0,WwRqopwVyvlzB,해리포터와 죽음의 성물 1,https://pedia.watcha.com/ko-KR/contents/m5ZXwL...,3.5
1,87Gv7eQBjxE6o,해리포터와 죽음의 성물 1,https://pedia.watcha.com/ko-KR/contents/m5ZXwL...,4.0
2,DgwxAeQYNxrMj,해리포터와 죽음의 성물 1,https://pedia.watcha.com/ko-KR/contents/m5ZXwL...,3.0
3,nkPvrJ84Dvgar,해리포터와 죽음의 성물 1,https://pedia.watcha.com/ko-KR/contents/m5ZXwL...,5.0
4,Qgy51MPlQ5Djk,해리포터와 죽음의 성물 1,https://pedia.watcha.com/ko-KR/contents/m5ZXwL...,4.0
...,...,...,...,...
10942,JgAx89MRDqLbO,블라인드,https://pedia.watcha.com/ko-KR/contents/m5e77o...,3.0
10943,nkPvrQAPJqgar,블라인드,https://pedia.watcha.com/ko-KR/contents/m5e77o...,3.0
10944,a9L5PwKN2vwg8,블라인드,https://pedia.watcha.com/ko-KR/contents/m5e77o...,3.0
10945,24BqEMAnOqrjn,블라인드,https://pedia.watcha.com/ko-KR/contents/m5e77o...,3.0


In [ ]:
# movie_ratings = pd.DataFrame()

In [ ]:
movie_ratings = pd.concat([movie_ratings, movie_rating])
movie_ratings.reset_index(drop=True, inplace=True)

In [ ]:
movie_ratings['movie_ID'] = movie_ratings['movie_ID'].map(lambda x: x.split('/')[-2])

In [ ]:
movie_ratings

,user,movie,movie_ID,rating
0,Pznx9j3wJq761,명량,mWzM9D5,2.0
1,8nPvyyWmovYo0,명량,mWzM9D5,1.0
2,DgwxAeQYNxrMj,명량,mWzM9D5,2.5
3,zNM5NrKAQ526j,명량,mWzM9D5,2.0
4,JgAx8QgkOqLbO,명량,mWzM9D5,1.5
...,...,...,...,...
27554,JgAx89MRDqLbO,블라인드,m5e77oO,3.0
27555,nkPvrQAPJqgar,블라인드,m5e77oO,3.0
27556,a9L5PwKN2vwg8,블라인드,m5e77oO,3.0
27557,24BqEMAnOqrjn,블라인드,m5e77oO,3.0


In [ ]:
len(movie_ratings['movie_ID'].unique())

497

In [ ]:
movie_ratings.to_csv('왓챠피디아 영화 평점.csv', index=False)

## 책 정보 크롤링

In [ ]:
user_ID = list(movie_ratings['user'].unique())
len(user_ID)

5583

In [ ]:
# 책 평점 크롤링
book_rating = pd.DataFrame()

for ID in tqdm(user_ID):
    book_url = 'https://pedia.watcha.com/ko-KR/users/' + ID + '/contents/books/ratings'
    driver.get(book_url) # 유저 책 페이지 접속
    time.sleep(1)

    # 끝까지 스크롤 다운
    scroll_location = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
        time.sleep(0.5)
        scroll_height = driver.execute_script('return document.body.scrollHeight')

        if scroll_location == scroll_height:
            break
        else:
            scroll_location = driver.execute_script('return document.body.scrollHeight')

    # 소스코드 BeautifulSoup에 입력하기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 사용자ID, 평점 수집 => 평점을 안매긴 사용자는 패스
    div_list = soup.find_all('div', {'class':'css-niy0za'})
    div_rating = soup.find_all('div', {'class':'css-m9i0qw'})
    book_title = list(map(lambda x: x.text, div_list))
    rating = list(map(lambda x:x.text.split('★')[-1], div_rating))

#     del_idx = -1

#     if None in div_rating:
#         del_idx = [i for i,j in enumerate(div_rating) if j == None]
#         for idx in del_idx[::-1]:
#             del div_rating[idx]

#     if del_idx != -1:
#         for idx in del_idx[::-1]:
#             del div_list[idx]

    book_list = list(map(lambda x: soup.find('a', {'title':x}).get('href'), book_title))
    book_ID = list(map(lambda x: x.split('/')[-1], book_list))

    # 데이터프레임 저장
    temp = pd.DataFrame()
    temp['book'] = book_title
    temp['book_ID'] = book_ID
    temp['user'] = ID
    temp['rating'] = rating
    book_rating = pd.concat([book_rating, temp])
    time.sleep(1)

book_rating = book_rating[['user','book','book_ID','rating']]
book_rating.reset_index(drop=True, inplace=True)
book_rating

100%|███████████████████████████████████████████████████████████████████████████| 5583/5583 [10:30:47<00:00,  6.78s/it]


,user,book,book_ID,rating
0,8nPvyyWmovYo0,해리 포터와 마법사의 돌,boYDGk4,3.0
1,8nPvyyWmovYo0,파리대왕,bowNK14,3.5
2,8nPvyyWmovYo0,지금 이 순간,boYOaR9,2.5
3,8nPvyyWmovYo0,식객,byzGpdW,4.0
4,8nPvyyWmovYo0,아웃라이어,bolJAN7,3.0
...,...,...,...,...
349752,17ovVzDWLvzyn,로빈슨 크루소,boO1nW9,4.0
349753,17ovVzDWLvzyn,이기는 습관,b4M5NEd,2.0
349754,17ovVzDWLvzyn,"멈추면, 비로소 보이는 것들",byaOv5x,2.5
349755,17ovVzDWLvzyn,우체부 프레드,bydRVNl,1.0


In [ ]:
book_ratings = pd.DataFrame()

In [ ]:
book_ratings = pd.concat([book_ratings, book_rating])
book_ratings.reset_index(drop=True, inplace=True)

In [ ]:
book_ratings

,user,book,book_ID,rating
0,8nPvyyWmovYo0,해리 포터와 마법사의 돌,boYDGk4,3.0
1,8nPvyyWmovYo0,파리대왕,bowNK14,3.5
2,8nPvyyWmovYo0,지금 이 순간,boYOaR9,2.5
3,8nPvyyWmovYo0,식객,byzGpdW,4.0
4,8nPvyyWmovYo0,아웃라이어,bolJAN7,3.0
...,...,...,...,...
349752,17ovVzDWLvzyn,로빈슨 크루소,boO1nW9,4.0
349753,17ovVzDWLvzyn,이기는 습관,b4M5NEd,2.0
349754,17ovVzDWLvzyn,"멈추면, 비로소 보이는 것들",byaOv5x,2.5
349755,17ovVzDWLvzyn,우체부 프레드,bydRVNl,1.0


In [ ]:
book_ratings.to_csv('왓챠피디아 책 평점.csv', index=False)